In [28]:
import os, shutil, glob
import PIL
from PIL import Image

#нужно скачать датасет https://data.mendeley.com/archiver/xnzhj3x8v4?version=2 вручную
#можно его же найти через поисковик по ключевым словам asphalt cracks dataset (скачивать version=2)
#фото асфальта без дефектов переложить в папку 0
#фото асфальта с дефектами переложить в папку 1
#я добавил asphalt.zip с этими картинками в проект

# функция do_5crop_and_flip делает 5 кропов углов и центра + для каждой из 5ти картинок выполняются опреации:
# horziontal flip
# vertical flip
# rotate_180 = horziontal flip + vertical flip
# за счет этого увеличиваем датасет в 20 раз

In [29]:
# for unix unzip ../dataset/asphalt.zip
!unzip ../dataset/asphalt.zip -d ../dataset/

Archive:  ../dataset/asphalt.zip
   creating: ../dataset/asphalt/
   creating: ../dataset/asphalt/0/
  inflating: ../dataset/asphalt/0/0.jpg  
  inflating: ../dataset/asphalt/0/1.jpg  
  inflating: ../dataset/asphalt/0/10.jpg  
  inflating: ../dataset/asphalt/0/100.jpg  
  inflating: ../dataset/asphalt/0/101.jpg  
  inflating: ../dataset/asphalt/0/102.jpg  
  inflating: ../dataset/asphalt/0/103.jpg  
  inflating: ../dataset/asphalt/0/104.jpg  
  inflating: ../dataset/asphalt/0/105.jpg  
  inflating: ../dataset/asphalt/0/106.jpg  
  inflating: ../dataset/asphalt/0/107.jpg  
  inflating: ../dataset/asphalt/0/108.jpg  
  inflating: ../dataset/asphalt/0/109.jpg  
  inflating: ../dataset/asphalt/0/11.jpg  
  inflating: ../dataset/asphalt/0/110.jpg  
  inflating: ../dataset/asphalt/0/111.jpg  
  inflating: ../dataset/asphalt/0/112.jpg  
  inflating: ../dataset/asphalt/0/113.jpg  
  inflating: ../dataset/asphalt/0/114.jpg  
  inflating: ../dataset/asphalt/0/115.jpg  
  inflating: ../dataset/a

  inflating: ../dataset/asphalt/1/118.jpg  
  inflating: ../dataset/asphalt/1/119.jpg  
  inflating: ../dataset/asphalt/1/12.jpg  
  inflating: ../dataset/asphalt/1/120.jpg  
  inflating: ../dataset/asphalt/1/121.jpg  
  inflating: ../dataset/asphalt/1/122.jpg  
  inflating: ../dataset/asphalt/1/123.jpg  
  inflating: ../dataset/asphalt/1/124.jpg  
  inflating: ../dataset/asphalt/1/125.jpg  
  inflating: ../dataset/asphalt/1/126.jpg  
  inflating: ../dataset/asphalt/1/127.jpg  
  inflating: ../dataset/asphalt/1/128.jpg  
  inflating: ../dataset/asphalt/1/129.jpg  
  inflating: ../dataset/asphalt/1/13.jpg  
  inflating: ../dataset/asphalt/1/130.jpg  
  inflating: ../dataset/asphalt/1/131.jpg  
  inflating: ../dataset/asphalt/1/132.jpg  
  inflating: ../dataset/asphalt/1/133.jpg  
  inflating: ../dataset/asphalt/1/134.jpg  
  inflating: ../dataset/asphalt/1/135.jpg  
  inflating: ../dataset/asphalt/1/136.jpg  
  inflating: ../dataset/asphalt/1/137.jpg  
  inflating: ../dataset/asphalt/1/

In [30]:
# make directories
!mkdir ../dataset/1
!mkdir ../dataset/0


def slash_index(img_path):
    for i in range(len(img_path)):
        if (img_path[-1-i] == '/'):
            return (len(img_path)-i-1)
    return 0

In [31]:


def get_coord_m10(w, h):
    minus = h//10
    return (0, 0, w, h-minus)


def get_coord_p10(w, h):
    minus = h//10
    return (0, minus, w, h)


def get_coord_c(w, h):
    return (0, 0, w, h)


def get_coord_cent(w, h):
    minus = (w-h)//2
    return (minus, 0, w-minus, h)


def get_coord_p50(w, h):
    minus = (w-h)//2
    return (minus-200, 0, w-minus-200, h)


def get_coord_m50(w, h):
    minus = (w-h)//2
    return (minus+200, 0, w-minus+200, h)


def get_coord_l(w, h):
    return (0, 0, w//2+1, h)


def get_coord_l_rect(w, h):
    minus = h//10
    return (0, 0, w//2+1, h-minus)


def save_flip_left_right(img1, outpath, final_name):
    img2 = img1.transpose(Image.FLIP_LEFT_RIGHT)
    img2 = img2.resize((224,224))
    img2.save(os.path.join(outpath, final_name))
    img2.close()


def save_flip_top_bottom(img1, outpath, final_name):
    img2 = img1.transpose(Image.FLIP_TOP_BOTTOM)
    img2 = img2.resize((224,224))
    img2.save(os.path.join(outpath, final_name))
    img2.close()


def save_rotate_180(img1, outpath, final_name):
    img2 = img1.transpose(Image.ROTATE_180)
    img2 = img2.resize((224,224))
    img2.save(os.path.join(outpath, final_name))
    img2.close()


def save_2flips_with_rotation(img1, outpath, n, name):
    save_rotate_180(img1, outpath, str(n+1500)+name)
    save_flip_top_bottom(img1, outpath, str(n+1000)+name)
    save_flip_left_right(img1, outpath, str(n+500)+name)


def save_ul2(img, name, n, outpath):
    wi, hi = img.size
    img1 = img.crop((0,0,wi-(wi//10),hi-(hi//10)))
    img2 = img1.resize((224,224))
    img2.save(os.path.join(outpath, str(n)+name))
    img2.close()
    save_2flips_with_rotation(img1, outpath, n, name)


def save_ur2(img, name, n, outpath):
    wi, hi = img.size
    img1 = img.crop(((wi//10),0,wi,hi-(hi//10)))
    img2 = img1.resize((224,224))
    img2.save(os.path.join(outpath, str(n)+name))
    img2.close()
    save_2flips_with_rotation(img1, outpath, n, name)


def save_mid2(img, name, n, outpath):
    wi, hi = img.size
    img1 = img.crop((wi//20,hi//20,wi-(wi//20),hi-(hi//20)))
    img2 = img1.resize((224,224))
    img2.save(os.path.join(outpath, str(n)+name))
    img2.close()
    save_2flips_with_rotation(img1, outpath, n, name)


def save_ll2(img, name, n, outpath):
    wi, hi = img.size
    img1 = img.crop((0,(hi//10),wi-(wi//10),hi))
    img2 = img1.resize((224,224))
    img2.save(os.path.join(outpath, str(n)+name))
    img2.close()
    save_2flips_with_rotation(img1, outpath, n, name)


def save_lr2(img, name, n, outpath):
    wi, hi = img.size
    img1 = img.crop((wi//10, hi//10, wi, hi))
    img2 = img1.resize((224,224))
    img2.save(os.path.join(outpath, str(n)+name))
    img2.close()
    save_2flips_with_rotation(img1, outpath, n, name)



In [32]:
def do_5crop_and_flip(imgpath, outpath, my_names):
    cnt = 0
    for name in my_names:
        img = Image.open(os.path.join(imgpath, name))
        wi, hi = img.size
        img1 = img
        save_ul2(img1, 'i_'+name, 0, outpath)
        save_ur2(img1, 'i_'+name, 1, outpath)
        save_mid2(img1, 'i_'+name, 2, outpath)
        save_ll2(img1, 'i_'+name, 3, outpath)
        save_lr2(img1, 'i_'+name, 4, outpath)
        img1.resize((224,224)).save(os.path.join(outpath, 'i'+name))
        img.close()
        cnt += 1
    print("done", cnt)



In [33]:
# функция do_5crop_and_flip делает 5 кропов углов и центра + для каждой из 5ти картинок выполняются опреации:
# horziontal flip
# vertical flip
# rotate_180 = horziontal flip + vertical flip
# за счет этого увеличиваем датасет в 20 раз

imgpath = '../dataset/asphalt/1'
outpath = '../dataset/1'

my_names = [fn[slash_index(fn)+1:] for fn in glob.glob(os.path.join(imgpath,'*.jpg'))]
names = sorted(my_names)
print("sorted name 99: ", names[99])
do_5crop_and_flip(imgpath, outpath, names)

path_to_clean_asph = '../dataset/asphalt/0'
out = "../dataset/0"
my_names = [fn[slash_index(fn)+1:] for fn in glob.glob(os.path.join(path_to_clean_asph,'*.jpg'))]
names = sorted(my_names)
print("name 90: ", names[90])
do_5crop_and_flip(path_to_clean_asph, out, names)


sorted name 99:  188.jpg
done 204
name 90:  18.jpg
done 196


In [34]:
#нужно скачать датасет https://data.mendeley.com/archiver/xnzhj3x8v4?version=2 вручную
#можно его же найти через поисковик по ключевым словам asphalt cracks dataset (скачивать version=2)
#фото асфальта без дефектов переложить в папку 0
#фото асфальта с дефектами переложить в папку 1
#я добавил asphalt.zip с этими картинками в проект

# функция do_5crop_and_flip делает 5 кропов углов и центра + для каждой из 5ти картинок выполняются опреации:
# horziontal flip
# vertical flip
# rotate_180 = horziontal flip + vertical flip
# за счет этого увеличиваем датасет в 20 раз

# for unix unzip ../dataset/asphalt.zip
!unzip ../dataset/luki.zip -d ../dataset/

Archive:  ../dataset/luki.zip
   creating: ../dataset/luki/
  inflating: ../dataset/luki/luk_0074.jpg  
  inflating: ../dataset/luki/luk_0075.jpg  
  inflating: ../dataset/luki/luk_0076.jpg  
  inflating: ../dataset/luki/luk_0077.jpg  
  inflating: ../dataset/luki/luk_0078.jpg  
  inflating: ../dataset/luki/luk_0079.jpg  
  inflating: ../dataset/luki/luk_0080.jpg  
  inflating: ../dataset/luki/luk_0081.jpg  
  inflating: ../dataset/luki/luk_0082.jpg  
  inflating: ../dataset/luki/luk_0083.jpg  
  inflating: ../dataset/luki/luk_0084.jpg  
  inflating: ../dataset/luki/luk_0085.jpg  
  inflating: ../dataset/luki/luk_0086.jpg  
  inflating: ../dataset/luki/luk_0087.jpg  
  inflating: ../dataset/luki/luk_0088.jpg  
  inflating: ../dataset/luki/luk_0089.jpg  
  inflating: ../dataset/luki/luk_0090.jpg  
  inflating: ../dataset/luki/luk_0091.jpg  
  inflating: ../dataset/luki/luk_0092.jpg  
  inflating: ../dataset/luki/luk_0093.jpg  
  inflating: ../dataset/luki/luk_0094.jpg  
  inflating: ../

  inflating: ../dataset/luki/luk_1480.jpg  
  inflating: ../dataset/luki/luk_1481.jpg  
  inflating: ../dataset/luki/luk_1482.jpg  
  inflating: ../dataset/luki/luk_1483.jpg  
  inflating: ../dataset/luki/luk_1484.jpg  
  inflating: ../dataset/luki/luk_1485.jpg  
  inflating: ../dataset/luki/luk_1486.jpg  
  inflating: ../dataset/luki/luk_1487.jpg  
  inflating: ../dataset/luki/luk_1488.jpg  
  inflating: ../dataset/luki/luk_1489.jpg  
  inflating: ../dataset/luki/luk_1490.jpg  
  inflating: ../dataset/luki/luk_1491.jpg  
  inflating: ../dataset/luki/luk_1492.jpg  
  inflating: ../dataset/luki/luk_1493.jpg  


In [35]:
# функция do_5crop_and_flip делает 5 кропов углов и центра + для каждой из 5ти картинок выполняются опреации:
# horziontal flip
# vertical flip
# rotate_180 = horziontal flip + vertical flip
# за счет этого увеличиваем датасет в 20 раз
!mkdir "../dataset/2"

imgpath = '../dataset/luki'
outpath = '../dataset/2'

my_names = [fn[slash_index(fn)+1:] for fn in glob.glob(os.path.join(imgpath,'*.jpg'))]
names = sorted(my_names)
print("sorted name 9: ", names[9])
do_5crop_and_flip(imgpath, outpath, names)


sorted name 9:  luk_0083.jpg
done 261


In [36]:
!mkdir ../dataset/train

In [37]:
!mkdir ../dataset/test
!mkdir ../dataset/test/0
!mkdir ../dataset/test/1
!mkdir ../dataset/test/2

!mv ../dataset/0/0i_11.jpg ../dataset/test/0/0i_11.jpg
!mv ../dataset/0/0i_22.jpg ../dataset/test/0/0i_22.jpg
!mv ../dataset/0/0i_33.jpg ../dataset/test/0/0i_33.jpg
!mv ../dataset/0/0i_44.jpg ../dataset/test/0/0i_44.jpg
!mv ../dataset/0/0i_55.jpg ../dataset/test/0/0i_55.jpg

!mv ../dataset/1/0i_11.jpg ../dataset/test/1/0i_11.jpg
!mv ../dataset/1/0i_22.jpg ../dataset/test/1/0i_22.jpg
!mv ../dataset/1/0i_33.jpg ../dataset/test/1/0i_33.jpg
!mv ../dataset/1/0i_44.jpg ../dataset/test/1/0i_44.jpg
!mv ../dataset/1/0i_55.jpg ../dataset/test/1/0i_55.jpg

!mv ../dataset/2/0i_luk_0074.jpg ../dataset/test/2/0i_luk_0074.jpg
!mv ../dataset/2/0i_luk_0075.jpg ../dataset/test/2/0i_luk_0075.jpg
!mv ../dataset/2/0i_luk_0076.jpg ../dataset/test/2/0i_luk_0076.jpg
!mv ../dataset/2/0i_luk_0077.jpg ../dataset/test/2/0i_luk_0077.jpg
!mv ../dataset/2/0i_luk_0078.jpg ../dataset/test/2/0i_luk_0078.jpg

!mv ../dataset/0 ../dataset/train/
!mv ../dataset/1 ../dataset/train/
!mv ../dataset/2 ../dataset/train/